In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [98]:
count_df = pd.read_csv('/content/drive/MyDrive/Penn_work/counts.cts', sep='\t')
engraftment_df = pd.read_csv('/content/drive/MyDrive/Penn_work/Engraftment_Sheet2.tsv', sep='\t')

In [99]:
count_df.head()

,Genes,14415-27,14415-20,14415-19,14415-33,14415-32,14415-23,14415-34,14415-29,14415-30.,...,14415-02,14415-03,14415-04,14415-07,14415-08,14415-09,14415-10,14415-11,14415-12,14415-13
0,A1BG,138,100,141,88,66,81,80,84,124,...,233,184,183,156,253,158,157,196,143,164
1,A1CF,2,1,2,2,0,1,2,1,5,...,3,5,0,8,2,5,0,4,0,1
2,A2M,3,1,3,8,8,10,3,7,21,...,1,12,3,4,5,6,2,6,0,3
3,A2ML1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,A3GALT2,1,2,4,1,0,2,1,0,0,...,1,6,0,1,4,2,0,1,0,0


In [100]:
#Removing patients for which we do not have engraftment data
unlabelled_patients = ['14415-30.', '14415-04', '14415-26']
cohort_2 = ['14415-22', '14415-12', '14415-13', '14415-14', '14415-16']

all_cohort_counts = count_df.drop(unlabelled_patients, axis=1)
two_cohort_counts = count_df.drop(unlabelled_patients + cohort_2, axis=1)
all_cohort_counts.head()

,Genes,14415-27,14415-20,14415-19,14415-33,14415-32,14415-23,14415-34,14415-29,14415-21,...,14415-17,14415-02,14415-03,14415-07,14415-08,14415-09,14415-10,14415-11,14415-12,14415-13
0,A1BG,138,100,141,88,66,81,80,84,78,...,245,233,184,156,253,158,157,196,143,164
1,A1CF,2,1,2,2,0,1,2,1,2,...,0,3,5,8,2,5,0,4,0,1
2,A2M,3,1,3,8,8,10,3,7,15,...,0,1,12,4,5,6,2,6,0,3
3,A2ML1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,A3GALT2,1,2,4,1,0,2,1,0,0,...,0,1,6,1,4,2,0,1,0,0


In [105]:
def process_engraftment(engraftment_df):
  new_engraftment = engraftment_df.copy()
  new_engraftment['Lowercase_Engraftment'] = new_engraftment.Engraftment.str.lower()
  new_engraftment['Binarized Engraftment'] = new_engraftment.Lowercase_Engraftment.eq('h').mul(1)
  new_engraftment.set_index('Patient', drop = True, inplace = True)
  return new_engraftment

processed_engraftment = process_engraftment(engraftment_df)

all_cohort_engraftment = processed_engraftment[~processed_engraftment.index.isin(unlabelled_patients)]
two_cohort_engraftment = processed_engraftment[~processed_engraftment.index.isin(unlabelled_patients + cohort_2)]
all_cohort_engraftment.head()

,Engraftment,Cohort,CAR_GIVEN,Lowercase_Engraftment,Binarized Engraftment
Patient,,,,,
14415-27,h,3,5.0,h,1
14415-20,h,3,5.0,h,1
14415-19,l,3,5.0,l,0
14415-33,h,3,5.0,h,1
14415-32,h,3,5.0,h,1


In [102]:
#Printing ratio of low engraftment in engraftment dataframe
def find_percent_zero(engraftment_df, type):
  num_zeroes = engraftment_df['Binarized Engraftment'][engraftment_df['Binarized Engraftment'] == 0].count()
  percent_zeroes = num_zeroes/len(engraftment_df)
  print(f'Ratio of {type} Patients with Low Engraftment: %0.3f' %percent_zeroes)

find_percent_zero(all_cohort_engraftment, 'All')
find_percent_zero(two_cohort_engraftment, 'Cohort 1 and 3')

Ratio of All Patients with Low Engraftment: 0.542
Ratio of Cohort 1 and 3 Patients with Low Engraftment: 0.526


In [103]:
#Standardizing by rows and transposing
def process_counts(input_df):
    temp_df = input_df.iloc[:,1:].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
    tranposed_data = temp_df.T
    tranposed_data.columns = list(input_df['Genes'])

    return tranposed_data

ProcessedCounts_all_cohort = process_counts(all_cohort_counts)
ProcessedCounts_two_cohort = process_counts(two_cohort_counts)

ProcessedCounts_all_cohort.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
14415-27,-0.264270,-0.314714,-0.314343,-0.315456,-0.315085,-0.310634,-0.315456,0.014655,0.007979,-0.315456,...,0.127412,0.615159,-0.276139,-0.194539,-0.069542,-0.309892,-0.072509,0.957881,0.406708,-0.034676
14415-20,-0.280881,-0.333951,-0.333951,-0.334487,-0.333415,-0.332343,-0.333951,0.028432,0.061668,-0.334487,...,0.165665,0.754806,-0.274448,-0.168306,-0.026784,-0.325374,-0.084679,0.948863,0.443885,-0.025711
14415-19,-0.262051,-0.303858,-0.303557,-0.304158,-0.303256,-0.304158,-0.304459,0.114511,-0.056927,-0.304459,...,0.201133,0.933203,-0.262051,-0.175730,-0.045197,-0.293030,-0.099636,0.705521,0.435431,0.018867
14415-33,-0.272915,-0.315597,-0.312619,-0.316590,-0.316094,-0.313612,-0.316590,0.034301,-0.022278,-0.316590,...,0.182201,0.625902,-0.266463,-0.158267,-0.052057,-0.310634,-0.081339,0.714245,0.327124,-0.033197
14415-32,-0.264946,-0.289851,-0.286833,-0.289851,-0.289851,-0.288719,-0.289851,-0.030236,-0.012878,-0.289851,...,0.143344,0.641820,-0.257022,-0.171742,-0.095517,-0.284946,-0.073631,1.117655,0.447109,-0.008350


In [139]:
#Dimensionality reduction and adding in engraftment data

def dim_reduction(prepped_data, engraftment_df):
  pca = PCA(n_components=len(prepped_data))
  reduced_data = pca.fit_transform(prepped_data)
  pca_df = pd.DataFrame(reduced_data)
  pca_df.index = prepped_data.index
  pca_df = pca_df.add_prefix('PCA_')

  final_df = pca_df.join(engraftment_df['Binarized Engraftment'])

  return final_df

FinalData_all_cohort = dim_reduction(ProcessedCounts_all_cohort, all_cohort_engraftment)
FinalData_two_cohort = dim_reduction(ProcessedCounts_two_cohort, two_cohort_engraftment)
FinalData_all_cohort.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,


,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,...,PCA_15,PCA_16,PCA_17,PCA_18,PCA_19,PCA_20,PCA_21,PCA_22,PCA_23,Binarized Engraftment
14415-27,-18.689825,0.079637,4.519172,2.008908,-4.920075,-1.012906,2.370325,-6.698862,1.225367,-1.274294,...,3.105196,1.221618,-1.860617,-1.006210,0.475015,-0.096045,-1.786261,-0.379458,6.132633e-14,1
14415-20,-13.724648,11.785317,9.384938,-0.231722,2.128080,-3.166537,3.703059,-2.028206,0.502449,1.148563,...,-2.853249,0.873270,-2.421866,-0.951219,0.375278,-1.901776,1.170050,0.725620,6.132633e-14,1
14415-19,9.324186,21.321186,0.508552,6.220917,-8.776135,-2.431148,-2.295982,-1.941151,3.964580,-5.084129,...,-2.034987,-0.904338,-1.085403,1.716591,-0.710112,0.208409,0.924869,1.308831,6.132633e-14,0
14415-33,-13.801762,9.192488,-0.440205,6.569910,1.234230,-5.187012,4.947837,-5.253199,-1.898007,5.608176,...,0.156851,1.109077,3.042866,3.153213,-2.032969,0.256493,-0.042695,-0.160832,6.132633e-14,1
14415-32,-18.204651,-5.495565,-5.626907,-4.608023,-6.923238,-0.102253,-7.522379,3.703321,1.651120,0.682308,...,1.049925,-2.083038,4.070229,0.891561,-1.534201,-1.056422,-0.090121,1.067889,6.132633e-14,1


In [140]:
def tune_logistic(final_data):
  model = LogisticRegression()
  solvers = ['newton-cg', 'lbfgs', 'liblinear']
  penalty = ['l2']
  c_values = [100, 10, 1.0, 0.1, 0.01]

  grid = dict(solver=solvers,penalty=penalty,C=c_values)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
  grid_result = grid_search.fit(final_data.iloc[:,:-1], final_data.iloc[:,-1])

  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

In [152]:
#Best params for all cohorts
tune_logistic(FinalData_all_cohort)

Best: 0.550000 using {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
0.444444 (0.286529) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.444444 (0.286529) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.483333 (0.328718) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.444444 (0.286529) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.444444 (0.286529) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.472222 (0.327966) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.455556 (0.301027) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.455556 (0.301027) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.461111 (0.315299) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.511111 (0.275322) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.511111 (0.275322) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.483333 (0.317251) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.522222 (0.287819) wi

In [153]:
#Best params for cohorts 1 and 3
tune_logistic(FinalData_two_cohort)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  UserWarning,


Best: 0.700000 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.700000 (0.355903) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.700000 (0.355903) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.650000 (0.368556) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.700000 (0.355903) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.700000 (0.355903) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.616667 (0.357849) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.700000 (0.355903) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.700000 (0.355903) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.616667 (0.333750) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.650000 (0.368556) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.650000 (0.368556) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.633333 (0.314466) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.616667 (0.357849) wit

In [172]:
def logistic_regression(final_data, penalty, C, solver, type):
  X_train, X_test, y_train, y_test = train_test_split(final_data.iloc[:,:-1], final_data.iloc[:,-1],
                                                      test_size=0.33, stratify = final_data.iloc[:,-1], random_state=42)
  
  LR_classifier = LogisticRegression(penalty = 'l2', C = 0.01, solver = 'liblinear', random_state=0)
  LR_classifier.fit(X_train, y_train)

  logistic_pred = LR_classifier.predict(X_test)
  print('Engraftment Predictions: ', logistic_pred)
  accuracy = np.count_nonzero(logistic_pred == y_test)/len(y_test)
  print(f'Accuracy for {type}: %0.2f' %accuracy, f'(Parameters: {penalty}, %0.2f, {solver})' %C)

In [173]:
#Tuning with best params for all cohorts
logistic_regression(FinalData_all_cohort, 'l2', 0.01, 'liblinear', 'All Patients')
logistic_regression(FinalData_two_cohort, 'l2', 0.01, 'liblinear', 'Cohorts 1 and 3')

Engraftment Predictions:  [0 0 1 0 1 1 0 1]
Accuracy for All Patients: 0.75 (Parameters: l2, 0.01, liblinear)
Engraftment Predictions:  [1 1 1 1 0 0 0]
Accuracy for Cohorts 1 and 3: 0.57 (Parameters: l2, 0.01, liblinear)


In [169]:
#Training with best params for cohorts 1 and 3

logistic_regression(FinalData_all_cohort, 'l2', 100, 'newton-cg', 'All Patients')
logistic_regression(FinalData_two_cohort, 'l2', 100, 'newton-cg', 'Cohorts 1 and 3')

Engraftment Predictions:  [0 0 1 0 1 1 0 1]
Accuracy for All Patients: 0.75 (Parameters: l2, 100.00, newton-cg)
Engraftment Predictions:  [1 1 1 1 0 0 0]
Accuracy for Cohorts 1 and 3: 0.57 (Parameters: l2, 100.00, newton-cg)


In [174]:
#Trying an autoencoder
from sklearn.preprocessing import MinMaxScaler

def autoencoder_logistic_regression(final_data, penalty, C, solver, type):
  X_train, X_test, y_train, y_test = train_test_split(final_data.iloc[:,:-1], final_data.iloc[:,-1],
                                                      test_size=0.33, stratify = final_data.iloc[:,-1], random_state=42)
  

  t = MinMaxScaler()
  t.fit(X_train)
  X_train = t.transform(X_train)
  X_test = t.transform(X_test)

  LR_classifier = LogisticRegression(penalty = 'l2', C = 0.01, solver = 'liblinear', random_state=0)
  
  LR_classifier.fit(X_train, y_train)
  y_pred = LR_classifier.predict(X_test)

  print('Engraftment Predictions: ', y_pred)
  accuracy = np.count_nonzero(y_pred == y_test)/len(y_test)
  print(f'Accuracy for {type}: %0.2f' %accuracy, f'(Parameters: {penalty}, %0.2f, {solver})' %C)

logistic_regression(FinalData_all_cohort, 'l2', 100, 'newton-cg', 'All Patients')

Engraftment Predictions:  [0 0 1 0 1 1 0 1]
Accuracy for All Patients: 0.75 (Parameters: l2, 100.00, newton-cg)


In [ ]:
#Visualization